# Code Description
This code is to perform embedding between visual features and class wise semantic attributes (length 85 per class) for AWA2 dataset.
The steps are:
1. use pretrained ResNet101 model (pretrained on ImageNet dataset) and remove final layers
2. Add additional layers and finally a dense layer of length 85, as each of the 50 classes have attributes of length 85
3. Train the model input is image of size 224, 224, 3 and output is of size 85
4. Evaluate the model for test seen and test unseen classes

In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import numpy as np
import scipy.io as sio
from numpy import expand_dims
from numpy import asarray
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from skimage.filters import gabor_kernel
from scipy import ndimage as nd        
import matplotlib
import matplotlib.pyplot as plt    
from numpy.linalg import norm
from keras.optimizers import SGD, Adam, Adagrad
import graphviz
import pydot

In [2]:
data_path = 'E:/Sushree/Dataset/'

img_path = os.path.join(data_path,'Animals_with_Attributes2/JPEGImages/')
print(img_path)

print(len(os.listdir(img_path)))

def get_imlist(path, option):
    if option == 'jpg':
        return[os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]
    
def prepare_dataset(img_path, width, height, option):
    folder_list = os.listdir(img_path)
    num_classes = len(folder_list)
    images = []
    labels = []
    for i in range(len(folder_list)):
        print(i)
        img_list = get_imlist(os.path.join(img_path, folder_list[i]), option)
        for j in range(len(img_list)):
            img = image.load_img(img_list[j], target_size = (width, height))
            img = np.array(img)
            #img = np.true_divide(img,[255.0], out = None)
            img = np.divide(np.subtract(img, np.mean(img)), np.std(img))
            x = image.img_to_array(img)
            images.append(x)
            labels.append(i)
    images = np.array(images, dtype = np.float32)                             
    labels = to_categorical(labels, num_classes)                             
    return images, labels 


width, height, ch = 224, 224, 3
images, labels = prepare_dataset(img_path, width, height,'jpg') 

print(images.shape)
print(len(labels), labels.shape)

E:/Sushree/Dataset/Animals_with_Attributes2/JPEGImages/
50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
(37322, 224, 224, 3)
37322 (37322, 50)


In [3]:

split_path = os.path.join(data_path,'data/xlsa17/data/AWA2/att_splits.mat')
matcontent = sio.loadmat(split_path)
train_loc = matcontent['train_loc'].squeeze() - 1

print(train_loc, len(train_loc))

train_images = images[train_loc]
print(train_images.shape)

train_labels = labels[train_loc]
print(len(train_labels), train_labels.shape)

val_loc = matcontent['val_loc'].squeeze() - 1

print(val_loc, len(val_loc))

val_images = images[val_loc]
print(val_images.shape)

val_labels = labels[val_loc]
print(len(val_labels), val_labels.shape)

[21482 11452 21448 ... 19615 10388  5160] 16187
(16187, 224, 224, 3)
16187 (16187, 50)
[12750 15467  8274 ... 28509 20799  7992] 7340
(7340, 224, 224, 3)
7340 (7340, 50)


In [5]:

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

batch_size = 16
train_gen = DataGenerator(train_images, train_labels, batch_size)   
val_gen = DataGenerator(val_images, val_labels, batch_size)

In [6]:
backbone_model = tf.keras.applications.resnet.ResNet101(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

backbone_model.summary()

# define new_model, with an additional dense(1024) layer after global average pooling  
x = backbone_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = BatchNormalization()(x)
#x = Dense(512, activation='relu')(x)
#x = Dropout(0.2)(x)
pred = Dense(50, activation='softmax')(x)

classifier_model = Model(inputs = backbone_model.input, outputs = pred)
classifier_model.summary()


for layer in classifier_model.layers:
    layer.trainable = True
    

sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
classifier_model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

train_summary = classifier_model.fit(train_gen, epochs = 300, verbose = 1, callbacks = None, validation_data = val_gen, 
                              shuffle = True, steps_per_epoch = len(train_gen)//batch_size, 
                              validation_steps = len(val_gen)//batch_size)
    

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                       

                                                                                                  
 conv2_block2_add (Add)         (None, None, None,   0           ['conv2_block1_out[0][0]',       
                                256)                              'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, None, None,   0           ['conv2_block2_add[0][0]']       
                                256)                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, None, None,   16448       ['conv2_block2_out[0][0]']       
                                64)                                                               
                                                                                                  
 conv2_blo

                                128)                                                              
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_2_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, None, None,   0          ['conv3_block2_2_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, None, None,   66048       ['conv3_block2_2_relu[0][0]']    
                                512)                                                              
          

 conv4_block1_1_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_1_conv[0][0]']    
 ization)                       256)                                                              
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, None, None,   0          ['conv4_block1_1_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, None, None,   590080      ['conv4_block1_1_relu[0][0]']    
                                256)                                                              
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_2_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block3_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block3_3_conv[0][0]']    
 ization)                       1024)                                                             
                                                                                                  
 conv4_block3_add (Add)         (None, None, None,   0           ['conv4_block2_out[0][0]',       
                                1024)                             'conv4_block3_3_bn[0][0]']      
                                                                                                  
 conv4_block3_out (Activation)  (None, None, None,   0           ['conv4_block3_add[0][0]']       
                                1024)                                                             
                                                                                                  
 conv4_blo

...

 on)                            256)                                                              
                                                                                                  
 conv4_block21_3_conv (Conv2D)  (None, None, None,   263168      ['conv4_block21_2_relu[0][0]']   
                                1024)                                                             
                                                                                                  
 conv4_block21_3_bn (BatchNorma  (None, None, None,   4096       ['conv4_block21_3_conv[0][0]']   
 lization)                      1024)                                                             
                                                                                                  
 conv4_block21_add (Add)        (None, None, None,   0           ['conv4_block20_out[0][0]',      
                                1024)                             'conv4_block21_3_bn[0][0]']     
          

 conv5_block1_2_conv (Conv2D)   (None, None, None,   2359808     ['conv5_block1_1_relu[0][0]']    
                                512)                                                              
                                                                                                  
 conv5_block1_2_bn (BatchNormal  (None, None, None,   2048       ['conv5_block1_2_conv[0][0]']    
 ization)                       512)                                                              
                                                                                                  
 conv5_block1_2_relu (Activatio  (None, None, None,   0          ['conv5_block1_2_bn[0][0]']      
 n)                             512)                                                              
                                                                                                  
 conv5_block1_0_conv (Conv2D)   (None, None, None,   2099200     ['conv4_block23_out[0][0]']      
          

                                                                                                  
 conv5_block3_out (Activation)  (None, None, None,   0           ['conv5_block3_add[0][0]']       
                                2048)                                                             
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['conv5_block3_out[0][0]']       
 alAveragePooling2D)                                                                              
                                                                                                  
 dense (Dense)                  (None, 1024)         2098176     ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
 dense_1 (

Epoch 46/300
63/63 [==============================] - 11s 168ms/step - loss: 0.1798 - accuracy: 0.9444 - val_loss: 12.8113 - val_accuracy: 0.0000e+00
Epoch 47/300
63/63 [==============================] - 11s 168ms/step - loss: 0.2471 - accuracy: 0.9222 - val_loss: 14.2050 - val_accuracy: 0.0000e+00
Epoch 48/300
63/63 [==============================] - 11s 168ms/step - loss: 0.2671 - accuracy: 0.9196 - val_loss: 13.0027 - val_accuracy: 0.0000e+00
Epoch 49/300
63/63 [==============================] - 11s 168ms/step - loss: 0.2088 - accuracy: 0.9395 - val_loss: 13.0787 - val_accuracy: 0.0000e+00
Epoch 50/300
63/63 [==============================] - 11s 168ms/step - loss: 0.1150 - accuracy: 0.9643 - val_loss: 14.8974 - val_accuracy: 0.0000e+00
Epoch 51/300
63/63 [==============================] - 11s 168ms/step - loss: 0.2108 - accuracy: 0.9444 - val_loss: 13.1510 - val_accuracy: 0.0000e+00
Epoch 52/300
63/63 [==============================] - 11s 168ms/step - loss: 0.2400 - accuracy: 0.93

KeyboardInterrupt: 

In [6]:
save_path = 'C:/Users/Admin/Sushree_Codes/Sush_2/Results/AWA2/ResNet101_finetuning/'
name = 'classifier_model_ResNet101_FT_1024_AWA2_300eph_sgd_cce_16bch_0.0001lr'
classifier_model.load_weights(save_path + 'Wt_' + name + '.h5')

# Results of multiclass classification

classifier_model_ResNet101_FT_1024_AWA2_100eph_sgd_cce_16bch_0.0001lr
Epoch 100/100
loss: 0.0535 - accuracy: 0.9846 - val_loss: 0.8877 - val_accuracy: 0.7986


classifier_model_ResNet101_FT_1024_AWA2_200eph_sgd_cce_16bch_0.0001lr
Epoch 200/200
loss: 0.0055 - accuracy: 0.9974 - val_loss: 0.7977 - val_accuracy: 0.8368


classifier_model_ResNet101_FT_1024_AWA2_300eph_sgd_cce_16bch_0.0001lr
Epoch 300/300
loss: 5.1633e-05 - accuracy: 1.0000 - val_loss: 0.9536 - val_accuracy: 0.8542


classifier_model_ResNet101_FT_1024bn_AWA2_200eph_sgd_cce_16bch_0.0001lr
Epoch 200/200
loss: 0.7452 - accuracy: 0.7911 - val_loss: 3.0020 - val_accuracy: 0.4201


classifier_model_ResNet101_FT_1024d0.2_AWA2_200eph_sgd_cce_16bch_0.0001lr
Epoch 200/200
loss: 0.0397 - accuracy: 0.9914 - val_loss: 1.0560 - val_accuracy: 0.7917


classifier_model_ResNet101_FT_1024_512_AWA2_200eph_sgd_cce_16bch_0.0001lr
Epoch 200/200
loss: 0.0602 - accuracy: 0.9812 - val_loss: 1.1172 - val_accuracy: 0.7917


classifier_model_ResNet101_FT_1024bn_512bn_AWA2_200eph_sgd_cce_16bch_0.0001lr
Epoch 200/200
loss: 1.4784 - accuracy: 0.5788 - val_loss: 2.6333 - val_accuracy: 0.4097


classifier_model_ResNet101_FT_1024d0.2_512d0.2_AWA2_200eph_sgd_cce_16bch_0.0001lr
Epoch 200/200
loss: 0.0435 - accuracy: 0.9897 - val_loss: 1.0904 - val_accuracy: 0.7986




In [7]:

attribute = matcontent['att'].T
print(attribute.shape)

attribute[attribute<0]=0

def get_imlist(path, option):
    if option == 'jpg':
        return[os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]
    
def prepare_labels(img_path, option):
    folder_list = os.listdir(img_path)
    num_classes = len(folder_list)
    labels = []
    for i in range(len(folder_list)):
        print(i)
        img_list = get_imlist(os.path.join(img_path, folder_list[i]), option)
        for j in range(len(img_list)):
            labels.append(i)
    return labels  

labels = prepare_labels(img_path, 'jpg')


train_labels_nocategorical = np.zeros(len(train_loc))
for i in range(len(train_loc)):
    train_labels_nocategorical[i] = labels[train_loc[i]]
print(len(train_labels_nocategorical), train_labels_nocategorical.shape)


train_attributes = np.zeros((len(train_loc), 85))
for i in range(len(train_loc)):
    train_attributes[i] = attribute[int(train_labels_nocategorical[i])]

print(train_attributes, train_attributes.shape)

val_labels_nocategorical = np.zeros(len(val_loc))
for i in range(len(val_loc)):
    val_labels_nocategorical[i] = labels[val_loc[i]]
print(len(val_labels_nocategorical), val_labels_nocategorical.shape)


val_attributes = np.zeros((len(val_loc), 85))
for i in range(len(val_loc)):
    val_attributes[i] = attribute[int(val_labels_nocategorical[i])]

print(val_attributes, val_attributes.shape)

(50, 85)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
16187 (16187,)
[[0.03588774 0.04482569 0.         ... 0.07673723 0.0732708  0.05016127]
 [0.1170179  0.05564648 0.         ... 0.09686609 0.0632034  0.01789046]
 [0.03588774 0.04482569 0.         ... 0.07673723 0.0732708  0.05016127]
 ...
 [0.2910046  0.10406716 0.00553889 ... 0.02219105 0.06454934 0.16698324]
 [0.01928112 0.         0.         ... 0.10161998 0.0093374  0.        ]
 [0.04707362 0.0159584  0.24440392 ... 0.09469377 0.03945877 0.01821735]] (16187, 85)
7340 (7340,)
[[0.13006453 0.02443221 0.         ... 0.03385142 0.11429366 0.04612166]
 [0.0084177  0.01262655 0.         ... 0.02104426 0.04138142 0.02316552]
 [0.16089413 0.         0.         ... 0.22788871 0.122741   0.05665286]
 ...
 [0.18650112 0.         0.         ... 0.0361739  0.03855809 0.00949565]
 [0.14428976 0.13846656 0.         ... 0.12204305 0.12762217 0.2491073

In [8]:
print(train_labels_nocategorical)
seen_classes = np.unique(train_labels_nocategorical)
print(seen_classes, seen_classes.shape)

[26. 14. 26. ... 11. 13.  8.]
[ 0.  2.  5.  6.  7.  8.  9. 10. 11. 13. 14. 15. 16. 18. 19. 20. 21. 23.
 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 38. 41. 42. 43. 44.
 45. 47. 48. 49.] (40,)


In [8]:

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 16
train_gen = DataGenerator(train_images, train_attributes, batch_size)   
val_gen = DataGenerator(val_images, val_attributes, batch_size)

In [9]:

for layer in classifier_model.layers:
    layer.trainable = True
    
x = classifier_model.layers[-4].output # shape should be (bs=None, 7, 7, 2048)
x = GlobalAveragePooling2D()(x)       # shape should be (bs=None, 2048)

x = Dense(1024, activation='relu')(x) # shape should be (bs=None, 1024)
#x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
#x = BatchNormalization()(x)
y = Dense(85, activation='linear')(x) # shape should be (bs=None, num_champti

    
embedding_model = Model(inputs = classifier_model.input, outputs=y)

    
embedding_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                         

                                                                                                  
 conv2_block2_add (Add)         (None, None, None,   0           ['conv2_block1_out[0][0]',       
                                256)                              'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, None, None,   0           ['conv2_block2_add[0][0]']       
                                256)                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, None, None,   16448       ['conv2_block2_out[0][0]']       
                                64)                                                               
                                                                                                  
 conv2_blo

                                128)                                                              
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_2_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, None, None,   0          ['conv3_block2_2_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, None, None,   66048       ['conv3_block2_2_relu[0][0]']    
                                512)                                                              
          

 conv4_block1_1_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_1_conv[0][0]']    
 ization)                       256)                                                              
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, None, None,   0          ['conv4_block1_1_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, None, None,   590080      ['conv4_block1_1_relu[0][0]']    
                                256)                                                              
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_2_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block3_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block3_3_conv[0][0]']    
 ization)                       1024)                                                             
                                                                                                  
 conv4_block3_add (Add)         (None, None, None,   0           ['conv4_block2_out[0][0]',       
                                1024)                             'conv4_block3_3_bn[0][0]']      
                                                                                                  
 conv4_block3_out (Activation)  (None, None, None,   0           ['conv4_block3_add[0][0]']       
                                1024)                                                             
                                                                                                  
 conv4_blo

...

 lization)                      1024)                                                             
                                                                                                  
 conv4_block14_add (Add)        (None, None, None,   0           ['conv4_block13_out[0][0]',      
                                1024)                             'conv4_block14_3_bn[0][0]']     
                                                                                                  
 conv4_block14_out (Activation)  (None, None, None,   0          ['conv4_block14_add[0][0]']      
                                1024)                                                             
                                                                                                  
 conv4_block15_1_conv (Conv2D)  (None, None, None,   262400      ['conv4_block14_out[0][0]']      
                                256)                                                              
          

 conv4_block17_2_relu (Activati  (None, None, None,   0          ['conv4_block17_2_bn[0][0]']     
 on)                            256)                                                              
                                                                                                  
 conv4_block17_3_conv (Conv2D)  (None, None, None,   263168      ['conv4_block17_2_relu[0][0]']   
                                1024)                                                             
                                                                                                  
 conv4_block17_3_bn (BatchNorma  (None, None, None,   4096       ['conv4_block17_3_conv[0][0]']   
 lization)                      1024)                                                             
                                                                                                  
 conv4_block17_add (Add)        (None, None, None,   0           ['conv4_block16_out[0][0]',      
          

                                                                                                  
 conv4_block20_2_conv (Conv2D)  (None, None, None,   590080      ['conv4_block20_1_relu[0][0]']   
                                256)                                                              
                                                                                                  
 conv4_block20_2_bn (BatchNorma  (None, None, None,   1024       ['conv4_block20_2_conv[0][0]']   
 lization)                      256)                                                              
                                                                                                  
 conv4_block20_2_relu (Activati  (None, None, None,   0          ['conv4_block20_2_bn[0][0]']     
 on)                            256)                                                              
                                                                                                  
 conv4_blo

                                256)                                                              
                                                                                                  
 conv4_block23_1_bn (BatchNorma  (None, None, None,   1024       ['conv4_block23_1_conv[0][0]']   
 lization)                      256)                                                              
                                                                                                  
 conv4_block23_1_relu (Activati  (None, None, None,   0          ['conv4_block23_1_bn[0][0]']     
 on)                            256)                                                              
                                                                                                  
 conv4_block23_2_conv (Conv2D)  (None, None, None,   590080      ['conv4_block23_1_relu[0][0]']   
                                256)                                                              
          

 conv5_block2_3_conv (Conv2D)   (None, None, None,   1050624     ['conv5_block2_2_relu[0][0]']    
                                2048)                                                             
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, None, None,   8192       ['conv5_block2_3_conv[0][0]']    
 ization)                       2048)                                                             
                                                                                                  
 conv5_block2_add (Add)         (None, None, None,   0           ['conv5_block1_out[0][0]',       
                                2048)                             'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_block2_out (Activation)  (None, None, None,   0           ['conv5_block2_add[0][0]']       
          

In [10]:
# the loss function as an inverse cosine distance
def cosine_loss(y, y_hat):
    # unit-normalize y and y_hat ()
    y = tf.math.l2_normalize(y, axis = 1)
    y_hat = tf.math.l2_normalize(y_hat, axis = 1)
    # cosine distance for normalized tensors
    loss = tf.compat.v1.losses.cosine_distance(y, y_hat, axis=1)
    return loss

In [11]:

adam = Adam(learning_rate = 0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#embedding_model.compile(adam, loss = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.AUTO), metrics = ['accuracy'])
embedding_model.compile(adam, loss = cosine_loss, metrics = ['accuracy'])

#CosineSimilarity is a number between -1 and 1. When it is a negative number between -1 and 0, 0 indicates orthogonality and 
#values closer to -1 indicate greater similarity. The values closer to 1 indicate greater dissimilarity. 

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_2/Results/'
name = 'embedding_model2_1024bn_512bn_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr'
file_path = save_path + 'bw_' + name + '.h5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = file_path,
    monitor = 'val_accuracy',
    mode = 'max',
    save_best_only=True)

train_summary = embedding_model.fit(train_gen, epochs = 200, verbose = 1, callbacks = [model_checkpoint_callback], validation_data = val_gen, 
                              shuffle = True, steps_per_epoch = len(train_gen)//batch_size, 
                              validation_steps = len(val_gen)//batch_size)

Epoch 1/200
63/63 [==============================] - 21s 201ms/step - loss: 0.7182 - accuracy: 0.0099 - val_loss: 0.5395 - val_accuracy: 0.0469
Epoch 2/200
63/63 [==============================] - 12s 183ms/step - loss: 0.4052 - accuracy: 0.0188 - val_loss: 0.3668 - val_accuracy: 0.0580
Epoch 3/200
63/63 [==============================] - 12s 185ms/step - loss: 0.3011 - accuracy: 0.0317 - val_loss: 0.3131 - val_accuracy: 0.0603
Epoch 4/200
63/63 [==============================] - 12s 186ms/step - loss: 0.2649 - accuracy: 0.0228 - val_loss: 0.2924 - val_accuracy: 0.0647
Epoch 5/200
63/63 [==============================] - 12s 186ms/step - loss: 0.2420 - accuracy: 0.0317 - val_loss: 0.2809 - val_accuracy: 0.0670
Epoch 6/200
63/63 [==============================] - 11s 167ms/step - loss: 0.2292 - accuracy: 0.0317 - val_loss: 0.2738 - val_accuracy: 0.0647
Epoch 7/200
63/63 [==============================] - 12s 188ms/step - loss: 0.2159 - accuracy: 0.0417 - val_loss: 0.2683 - val_accuracy:

63/63 [==============================] - 11s 169ms/step - loss: 0.0791 - accuracy: 0.3313 - val_loss: 0.2939 - val_accuracy: 0.0938
Epoch 58/200
63/63 [==============================] - 11s 169ms/step - loss: 0.0769 - accuracy: 0.3363 - val_loss: 0.2948 - val_accuracy: 0.0915
Epoch 59/200
63/63 [==============================] - 11s 169ms/step - loss: 0.0761 - accuracy: 0.3254 - val_loss: 0.2956 - val_accuracy: 0.0938
Epoch 60/200
63/63 [==============================] - 11s 169ms/step - loss: 0.0763 - accuracy: 0.3085 - val_loss: 0.2964 - val_accuracy: 0.0938
Epoch 61/200
63/63 [==============================] - 11s 169ms/step - loss: 0.0737 - accuracy: 0.3105 - val_loss: 0.2963 - val_accuracy: 0.0915
Epoch 62/200
63/63 [==============================] - 11s 169ms/step - loss: 0.0702 - accuracy: 0.3621 - val_loss: 0.2965 - val_accuracy: 0.0915
Epoch 63/200
63/63 [==============================] - 11s 169ms/step - loss: 0.0761 - accuracy: 0.3393 - val_loss: 0.2958 - val_accuracy: 0.087

KeyboardInterrupt: 

In [ ]:

embedding_model.save_weights(save_path + 'Wt_' + name + '.h5')

print(train_summary.history.keys())

plt.plot(train_summary.history['accuracy'])
plt.plot(train_summary.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.show()
plt.savefig(save_path + 'Acc_' + name + '.png')

plt.plot(train_summary.history['loss'])
plt.plot(train_summary.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.show()
plt.savefig(save_path + 'Loss_' + name + '.png')

In [12]:
test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
print(test_seen_loc, len(test_seen_loc))

test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1
print(test_unseen_loc, len(test_unseen_loc))

test_seen_images = images[test_seen_loc]
print(test_seen_images.shape)

test_unseen_images = images[test_unseen_loc]
print(test_unseen_images.shape)

test_seen_labels_nocategorical = np.zeros(len(test_seen_loc))
for i in range(len(test_seen_loc)):
    test_seen_labels_nocategorical[i] = labels[test_seen_loc[i]]
print(len(test_seen_labels_nocategorical), test_seen_labels_nocategorical.shape)

test_seen_attributes = np.zeros((len(test_seen_loc), 85))
for i in range(len(test_seen_loc)):
    test_seen_attributes[i] = attribute[int(test_seen_labels_nocategorical[i])]

print(test_seen_attributes, test_seen_attributes.shape)

test_unseen_labels_nocategorical = np.zeros(len(test_unseen_loc))
for i in range(len(test_unseen_loc)):
    test_unseen_labels_nocategorical[i] = labels[test_unseen_loc[i]]
print(len(test_unseen_labels_nocategorical), test_unseen_labels_nocategorical.shape)

test_unseen_attributes = np.zeros((len(test_unseen_loc), 85))
for i in range(len(test_unseen_loc)):
    test_unseen_attributes[i] = attribute[int(test_unseen_labels_nocategorical[i])]

print(test_unseen_attributes, test_unseen_attributes.shape)

[11060  6270 17297 ...  4013 20614 33201] 5882
[ 1046  1047  1048 ... 35288 35289 35290] 7913
(5882, 224, 224, 3)
(7913, 224, 224, 3)
5882 (5882,)
[[0.1170179  0.05564648 0.         ... 0.09686609 0.0632034  0.01789046]
 [0.18491055 0.07733939 0.04019938 ... 0.15638939 0.05664756 0.27484925]
 [0.         0.00507555 0.         ... 0.15675628 0.09070911 0.01425057]
 ...
 [0.1376261  0.05019528 0.01580456 ... 0.11097961 0.05224987 0.2166805 ]
 [0.14428976 0.13846656 0.         ... 0.12204305 0.12762217 0.24910731]
 [0.03021446 0.28891062 0.00755361 ... 0.14729547 0.05855561 0.01510723]] (5882, 85)
7913 (7913,)
[[0.12045618 0.00426584 0.         ... 0.17996306 0.0618086  0.03495531]
 [0.12045618 0.00426584 0.         ... 0.17996306 0.0618086  0.03495531]
 [0.12045618 0.00426584 0.         ... 0.17996306 0.0618086  0.03495531]
 ...
 [0.06218048 0.01590817 0.         ... 0.03693204 0.1114562  0.06188344]
 [0.06218048 0.01590817 0.         ... 0.03693204 0.1114562  0.06188344]
 [0.06218048 0.

In [13]:
test_seen_gen = DataGenerator(test_seen_images, test_seen_attributes, batch_size)
embedding_model.evaluate(test_seen_gen, steps = len(test_seen_gen)//batch_size, verbose = 1)

test_unseen_gen = DataGenerator(test_unseen_images, test_unseen_attributes, batch_size)
embedding_model.evaluate(test_unseen_gen, steps = len(test_unseen_gen)//batch_size, verbose = 1)

30/30 [==============================] - 1s 44ms/step - loss: 0.3200 - accuracy: 0.0437


[0.3200069069862366, 0.04374999925494194]

In [ ]:
test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
print(test_seen_loc, len(test_seen_loc))

test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1
print(test_unseen_loc, len(test_unseen_loc))

test_seen_images = images[test_seen_loc]
print(test_seen_images.shape)

test_unseen_images = images[test_unseen_loc]
print(test_unseen_images.shape)

test_seen_labels_nocategorical = np.zeros(len(test_seen_loc))
for i in range(len(test_seen_loc)):
    test_seen_labels_nocategorical[i] = labels[test_seen_loc[i]]
print(len(test_seen_labels_nocategorical), test_seen_labels_nocategorical.shape)

test_seen_attributes = np.zeros((len(test_seen_loc), 85))
for i in range(len(test_seen_loc)):
    test_seen_attributes[i] = attribute[int(test_seen_labels_nocategorical[i])]

print(test_seen_attributes, test_seen_attributes.shape)

test_unseen_labels_nocategorical = np.zeros(len(test_unseen_loc))
for i in range(len(test_unseen_loc)):
    test_unseen_labels_nocategorical[i] = labels[test_unseen_loc[i]]
print(len(test_unseen_labels_nocategorical), test_unseen_labels_nocategorical.shape)

test_unseen_attributes = np.zeros((len(test_unseen_loc), 85))
for i in range(len(test_unseen_loc)):
    test_unseen_attributes[i] = attribute[int(test_unseen_labels_nocategorical[i])]

print(test_unseen_attributes, test_unseen_attributes.shape)

In [ ]:
embedding_model.load_weights(save_path + 'bw_' + name + '.h5')

test_seen_gen = DataGenerator(test_seen_images, test_seen_attributes, batch_size)
embedding_model.evaluate(test_seen_gen, steps = len(test_seen_gen)//batch_size, verbose = 1)

test_unseen_gen = DataGenerator(test_unseen_images, test_unseen_attributes, batch_size)
embedding_model.evaluate(test_unseen_gen, steps = len(test_unseen_gen)//batch_size, verbose = 1)

In [ ]:
#cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
#y_true = [[0., 1.], [1., 1.]]
#y_pred = [[1., 0.], [1., 1.]]
#1 + cosine_loss(y_true, y_pred).numpy()

# Results

embedding_model_1024bn_RnoFT_C1024noFT_AWA2_200eph_adam_cosine_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (not finetuned), Classifier model: GAP, dense 1024 (trained for 100 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, batchnorm, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss (-1 to 1)]
Total params: 44,847,573, Trainable params: 44,740,181, Non-trainable params: 107,392

training:
Epoch 100/100
loss: -0.7888 - accuracy: 0.3014 - val_loss: -0.8337 - val_accuracy: 0.2917

test seen: loss: -0.8133 - accuracy: 0.2826
test unseen: loss: -0.5647 - accuracy: 0.0375


visualFT_to_attribute_85_embedding_1024_AWA2_200eph_adam_cosine_16bch_0.0001lr
Total params: 44,847,573, Trainable params: 44,740,181, Non-trainable params: 107,392
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 100 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, batchnorm, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss (-1 to 1)]

training:
Epoch 200/200
loss: -0.8460 - accuracy: 0.3930 - val_loss: -0.8431 - val_accuracy: 0.3611

test seen: loss: -0.8562 - accuracy: 0.3614
test unseen: loss: -0.4986 - accuracy: 0.0979

With best weights
test seen: loss: -0.8468 - accuracy: 0.3750
test unseen: loss: -0.4667 - accuracy: 0.0979


visualFT_to_attribute_85_embedding_noFT_1024_AWA2_200eph_adam_cosine_16bch_0.0001lr
Total params: 44,847,573, Trainable params: 2,187,349, Non-trainable params: 42,660,224
[Pretrained ResNet101 is finetuned fully for multiclass classification, attributes of length 85 are obtained, negative values are made zero for the entire dataset, additional layers in the embedding network: (Dense 1024, batchnorm), weights of new_model are not updated, adam optimizer, lr = 0.0001, cosine loss (-1 to 1)]

training:
Epoch 200/200
loss: -0.8425 - accuracy: 0.3168 - val_loss: -0.8637 - val_accuracy: 0.2986

test seen: loss: -0.8749 - accuracy: 0.2935
test unseen: loss: -0.5484 - accuracy: 0.0479

With best weights
test seen: loss: -0.8749 - accuracy: 0.2935
test unseen: loss: -0.5484 - accuracy: 0.0479


embedding_model_1024bn_RFT_C1024_300FT_AWA2_200eph_adam_cosine_16bch_0.0001lr
Total params: 44,847,573, Trainable params: 44,740,181, Non-trainable params: 107,392
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, batchnorm, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss (-1 to 1)]

training:
Epoch 200/200
loss: -0.8437 - accuracy: 0.3870 - val_loss: -0.8507 - val_accuracy: 0.3264

test seen: loss: -0.8443 - accuracy: 0.3614
test unseen: loss: -0.6614 - accuracy: 0.0917

With best weights
test seen: loss: -0.8439 - accuracy: 0.3533
test unseen: loss: -0.6664 - accuracy: 0.0750


embedding_model_1024_RFT_C1024_300FT_AWA2_200eph_adam_cosine_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss (-1 to 1)]

Epoch 200/200
loss: -0.9721 - accuracy: 0.5497 - val_loss: -0.9100 - val_accuracy: 0.4271

test seen: loss: -0.9064 - accuracy: 0.4728
test unseen: loss: -0.6584 - accuracy: 0.0688

With best weights
test seen: loss: -0.9043 - accuracy: 0.4728
test unseen: loss: -0.6581 - accuracy: 0.0729


embedding_model_1024_512_RFT_C1024_300FT_AWA2_200eph_adam_cosine_16bch_0.0001lr
Total params: 45,324,757, Trainable params: 45,219,413, Non-trainable params: 105,344
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, Dense 512, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss (-1 to 1)]

training:
Epoch 200/200
loss: -0.9845 - accuracy: 0.6182 - val_loss: -0.9202 - val_accuracy: 0.4757

test seen: loss: -0.9144 - accuracy: 0.5353
test unseen: loss: -0.6623 - accuracy: 0.0521

With best weights
test seen: loss: -0.9115 - accuracy: 0.5299
test unseen: loss: -0.6622 - accuracy: 0.0479


embedding_model_1024_512_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, Dense 512, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss modified (0 to 1)]

training:
Epoch 200/200
loss: 0.0069 - accuracy: 0.7537 - val_loss: 0.0871 - val_accuracy: 0.5660

test seen: loss: 0.0914 - accuracy: 0.5978
test unseen: loss: 0.3411 - accuracy: 0.0917

With best weights
test seen: loss: 0.0863 - accuracy: 0.5924
test unseen: loss: 0.3412 - accuracy: 0.0938


embedding_model_bn1024_bn512_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, batch norm, Dense 1024, batch norm, Dense 512, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss modified (0 to 1)]

training:
Epoch 200/200
loss: 0.0155 - accuracy: 0.6293 - val_loss: 0.0899 - val_accuracy: 0.5035

test seen: loss: 0.0938 - accuracy: 0.5543
test unseen: loss: 0.3230 - accuracy: 0.0354

With best weights
test seen: loss: 0.0945 - accuracy: 0.5516
test unseen: loss: 0.3254 - accuracy: 0.0354


embedding_model_1024bn_512bn_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, batch norm, Dense 512, batch norm, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss modified (0 to 1)]

training:
Epoch 200/200
loss: 0.1281 - accuracy: 0.4204 - val_loss: 0.1271 - val_accuracy: 0.4062

test seen: loss: 0.1325 - accuracy: 0.4212
test unseen: loss: 0.3265 - accuracy: 0.0500

With best weights
test seen: loss: 0.1330 - accuracy: 0.4076
test unseen: loss: 0.3254 - accuracy: 0.0458


embedding_model_d0.21024_do.2512_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, drop0.2, Dense 1024, drop0.2, Dense 512, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss modified (0 to 1)]

training:
Epoch 200/200
loss: 0.0660 - accuracy: 0.2809 - val_loss: 0.0831 - val_accuracy: 0.2986

test seen: loss: 0.0792 - accuracy: 0.3315
test unseen: loss: 0.3188 - accuracy: 0.0979

With best weights
test seen: loss: 0.0792 - accuracy: 0.3315
test unseen: loss: 0.3188 - accuracy: 0.0979


embedding_model_1024do.2_512d0.2_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, drop0.2, Dense 512, drop0.2, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss modified (0 to 1)]

training:
Epoch 200/200
loss: 0.0638 - accuracy: 0.3091 - val_loss: 0.0703 - val_accuracy: 0.3646

test seen: loss: 0.0697 - accuracy: 0.3397
test unseen: loss: 0.3111 - accuracy: 0.0646

With best weights
test seen: loss: 0.0698 - accuracy: 0.3342
test unseen: loss: 0.3107 - accuracy: 0.0708


embedding_model_1024_512_128_RFT_C1024_300FT_AWA2_200eph_adam_cosine_16bch_0.0001lr
Total params: 45,357,781, Trainable params: 45,252,437, Non-trainable params: 105,344
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, Dense 512, Dense 128, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss (-1 to 1)]

Epoch 200/200
loss: -0.9842 - accuracy: 0.5981 - val_loss: -0.9208 - val_accuracy: 0.4479

test seen: loss: -0.9153 - accuracy: 0.5054
test unseen: loss: -0.6588 - accuracy: 0.0646

With best weights
test seen: loss: -0.9139 - accuracy: 0.4946
test unseen: loss: -0.6608 - accuracy: 0.0604


embedding_model_1024_512_128_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr
[Backbone model: Pretrained ResNet101 (finetuned fully for multiclass classification), Classifier model: GAP, dense 1024 (trained for 300 epochs), attributes of length 85 are obtained, negative values are made zero for the entire dataset, Embedding model: GAP, Dense 1024, Dense 512, Dense 128, (classifier_model finetuned), adam optimizer, lr = 0.0001, cosine loss modified (0 to 1)]

Epoch 200/200
loss: 0.0142 - accuracy: 0.6498 - val_loss: 0.0819 - val_accuracy: 0.5347

test seen: loss: 0.0847 - accuracy: 0.5190
test unseen: loss: 0.3455 - accuracy: 0.0750

With best weights
test seen: loss: 0.0846 - accuracy: 0.5027
test unseen: loss: 0.3454 - accuracy: 0.0708




In [11]:
adam = Adam(learning_rate = 0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#embedding_model.compile(adam, loss = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.AUTO), metrics = ['accuracy'])
embedding_model.compile(adam, loss = cosine_loss, metrics = ['accuracy'])

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_2/Results/'
name = 'embedding_model_1024_512_RFT_C1024_300FT_AWA2_200eph_adam_cosine2_16bch_0.0001lr'
file_path = save_path + 'bw_' + name + '.h5'



In [14]:

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

batch_size = 16
train_gen = DataGenerator(train_attributes, train_labels, batch_size)   
val_gen = DataGenerator(val_attributes, val_labels, batch_size)

In [15]:
def classifier(embedding_size):

    inputs = Input(shape=embedding_size)
    output = Dense(50, name="output", activation='softmax')(inputs)
    return Model(inputs, output, name="classifier")

In [16]:
clssifier_net = classifier(85)
clssifier_net.summary()

Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 85)]              0         
                                                                 
 output (Dense)              (None, 50)                4300      
                                                                 
Total params: 4,300
Trainable params: 4,300
Non-trainable params: 0
_________________________________________________________________


In [17]:
sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
clssifier_net.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

train_summary = clssifier_net.fit(train_gen, epochs = 100, verbose = 1, callbacks = None, validation_data = val_gen, 
                              shuffle = True, steps_per_epoch = len(train_gen)//batch_size, 
                              validation_steps = len(val_gen)//batch_size)

Epoch 1/100
63/63 [==============================] - 0s 4ms/step - loss: 3.8136 - accuracy: 0.0565 - val_loss: 4.1203 - val_accuracy: 0.0000e+00
Epoch 2/100
63/63 [==============================] - 0s 3ms/step - loss: 3.5413 - accuracy: 0.3224 - val_loss: 4.3214 - val_accuracy: 0.0000e+00
Epoch 3/100
63/63 [==============================] - 0s 3ms/step - loss: 3.3227 - accuracy: 0.4226 - val_loss: 4.5088 - val_accuracy: 0.0000e+00
Epoch 4/100
63/63 [==============================] - 0s 3ms/step - loss: 3.1509 - accuracy: 0.4187 - val_loss: 4.6752 - val_accuracy: 0.0000e+00
Epoch 5/100
63/63 [==============================] - 0s 3ms/step - loss: 3.0060 - accuracy: 0.4712 - val_loss: 4.8212 - val_accuracy: 0.0000e+00
Epoch 6/100
63/63 [==============================] - 0s 3ms/step - loss: 2.8663 - accuracy: 0.4831 - val_loss: 4.9547 - val_accuracy: 0.0000e+00
Epoch 7/100
63/63 [==============================] - 0s 3ms/step - loss: 2.7731 - accuracy: 0.4524 - val_loss: 5.0731 - val_accura

63/63 [==============================] - 0s 3ms/step - loss: 0.7888 - accuracy: 0.9484 - val_loss: 7.0159 - val_accuracy: 0.0000e+00
Epoch 58/100
63/63 [==============================] - 0s 3ms/step - loss: 0.7775 - accuracy: 0.9325 - val_loss: 7.0302 - val_accuracy: 0.0000e+00
Epoch 59/100
63/63 [==============================] - 0s 3ms/step - loss: 0.7584 - accuracy: 0.9454 - val_loss: 7.0453 - val_accuracy: 0.0000e+00
Epoch 60/100
63/63 [==============================] - 0s 3ms/step - loss: 0.7566 - accuracy: 0.9435 - val_loss: 7.0622 - val_accuracy: 0.0000e+00
Epoch 61/100
63/63 [==============================] - 0s 3ms/step - loss: 0.7388 - accuracy: 0.9435 - val_loss: 7.0800 - val_accuracy: 0.0000e+00
Epoch 62/100
63/63 [==============================] - 0s 3ms/step - loss: 0.7042 - accuracy: 0.9385 - val_loss: 7.0969 - val_accuracy: 0.0000e+00
Epoch 63/100
63/63 [==============================] - 0s 3ms/step - loss: 0.7135 - accuracy: 0.9375 - val_loss: 7.1151 - val_accuracy: 0.

In [19]:
test_seen_output = embedding_model.predict(test_seen_images, steps = len(test_seen_images)//batch_size, verbose = 1)

print(test_seen_output.shape)

test_unseen_output = embedding_model.predict(test_unseen_images, steps = len(test_unseen_images)//batch_size, verbose = 1)

print(test_unseen_output.shape)

367/367 [==============================] - 14s 35ms/step
(5882, 85)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [20]:
test_seen_gen = DataGenerator(test_seen_output, test_seen_labels, batch_size)
clssifier_net.evaluate(test_seen_gen, steps = len(test_seen_gen)//batch_size, verbose = 1)

test_unseen_gen = DataGenerator(test_unseen_output, test_unseen_labels, batch_size)
clssifier_net.evaluate(test_unseen_gen, steps = len(test_unseen_gen)//batch_size, verbose = 1)

NameError: name 'test_seen_labels' is not defined